## 파이토치 기본

### 파이토치 공식 튜토리얼
- https://tutorials.pytorch.kr/

### 파이토치 패키지(API)
- `torch` : 메인 네임스페이스. 텐서(기본단위) 등 수학함수 포함. Numpy와 유사한 구조
- `torch.autograd` : 자동미분을 위한 함수들 포함. 컨텍스트 매니저, 기반클래스 포함
- `torch.nn` / `torch.nn.functional` : neuralnetwork. 신경망 구축을 휘한 데이터 구조, 레이어등이 정의되어 있는 모듈. ReLU 등도 포함
- `torch.optim` : SGD(확률적 경사 하강법) 중심 파라미터 최적화 알고리즘 포함
- `torch.utils` : SGD 반복 연산 시 배치용 유틸리티 함수 포함
- torch.multiprocessing : 파이토치용 병렬프로세싱 환경을 안전하게 다루기 위한 모듈

### 텐서
- 파이토치 기본단위
    - 1차원 배열 - 백터
    - 2차원 배열 - 행렬(매트릭스)
    - 3차원 배열 이상 - 텐서

- <img src="../image/ml0018.png" width="700">

- <img src="../image/ml0019.png" width="700">

In [4]:
import torch 
import numpy as np

In [5]:
t1 = torch.tensor([1.0, 2.0, 3.0])
t1

tensor([1., 2., 3.])

In [6]:
# 텐서의 크기, 텐서 자료형, 자료형+전체타입, 어느 디바이스에서 사용중
print(t1.shape, t1.type, t1.type(), t1.device)

torch.Size([3]) <built-in method type of Tensor object at 0x000001E26CF2F890> torch.FloatTensor cpu


In [7]:
n1 = np.array([1.0, 2.0, 3.0])
n1

array([1., 2., 3.])

In [8]:
print(n1.shape, n1.dtype)

(3,) float64


In [9]:
## numpy에서 1 or 0으로 초기화되는 배열 만듦
n2 = np.ones([2, 3])
n2

array([[1., 1., 1.],
       [1., 1., 1.]])

In [10]:
n3 = np.zeros([2, 3])
n3

array([[0., 0., 0.],
       [0., 0., 0.]])

In [11]:
t2 = torch.ones(2, 3)
t2

tensor([[1., 1., 1.],
        [1., 1., 1.]])

In [12]:
t3  = torch.zeros(2, 3)
t3

tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [13]:
# numpy로 만든 배열을 텐서로 변환
t4 = torch.from_numpy(n3)
t4

tensor([[0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

- 넘파이와 파이토치 텐서간의 형변환이 아주 쉽다

In [14]:
t5 = torch.ones(3, 3, 3)
t5

tensor([[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]])

In [15]:
print(t5.shape, t5.dtype, t5.type(), t5.device)
print(t5.size())

torch.Size([3, 3, 3]) torch.float32 torch.FloatTensor cpu
torch.Size([3, 3, 3])


In [16]:
# 랜덤값, randn()은 정규화
t6 = torch.rand(2, 3)
t6

tensor([[0.3993, 0.9912, 0.6593],
        [0.3610, 0.0181, 0.3665]])

In [17]:
# cpu에 있는 텐서를 cuda로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [18]:
# t5(cpu)에 있는 텐서를 cuda로 변수명을 다르게 하면 복사, 변수명을 똑같이 하면 이동
t5 = t5.to(device=device)

In [19]:
print(t5.shape, t5.dtype, t5.type(), t5.device)

torch.Size([3, 3, 3]) torch.float32 torch.cuda.FloatTensor cuda:0


- cuda(GPU 사용하는 것) : 0(그래픽카드가 다수일 수 있음)
- 파이토치의 텐서사용 == 넘파이 배열 사용

### GPU 사용법

In [20]:
import torch.nn as nn

# 신경망 모델 생성 - 3개의 입력을 넣어서 하나의 출력을 내는 신경망
model = torch.nn.Linear(3, 1)

sample_input = torch.tensor([[1.0, 2.0, 3.0]])
output = model(sample_input) 

output

tensor([[-0.8242]], grad_fn=<AddmmBackward0>)

In [21]:
# CUDA 사용
import torch.nn as nn

# 신경망 모델 생성 - 3개의 입력을 넣어서 하나의 출력을 내는 신경망
model = torch.nn.Linear(3, 1)
model.to(device)

sample_input = torch.tensor([[1.0, 2.0, 3.0]]).to(device)
output = model(sample_input) 

output

tensor([[-0.6837]], device='cuda:0', grad_fn=<AddmmBackward0>)

#### 기본구조 추가

### PyTorch 모델 학습
- torch.nn.Module 로 신경망 모델 만들기 - 텐서플로우(함수)와 차이점 확인
- 손실 함수, 옵티마이저 설정 학습
- 학습 루프 기본 구조 학습

In [22]:
# 파이토치 모듈 로드
import torch
import torch.nn as nn
import torch.nn.functional as F    # 보통 F로 사용

# 모델 선언
class SimpleNet(nn.Module):     # nn.Module 클래스를 상속해서 클래스 생성
    def __init__(self):
        super(SimpleNet, self).__init__()
        # Linear : Dense Layer(케라스의 Dense()와 동일)
        self.fc1 = nn.Linear(4, 16)     # input: 4개 입력, output: 16개 뉴런
        self.fc2 = nn.Linear(16, 3)     # input: 16개 입력, output: 3개 클래스(3가지 분류 예측)
    
    def forward(self, x):           # 실제 연산이 수행되는 함수(자동 호출)
        x = F.relu(self.fc1(x))     # 첫번째 레이어 통과 - 활성함수 relu(시그모이드, 소프트맥스, 렐루 모두 존재)
        x = self.fc2(x)             # 두번째 레이어 통과 - 출력층(필요시 softmax, loss함수 사용)
        return x

- 텐서플로우(케라스)는 중간층에서 입력값을 설정하지 않아도 됨
- 파이토치는 입력값, 출력값을 모두 설정해야 함

#### 모델 생성

In [23]:
# 모델확인
model = SimpleNet()
model

SimpleNet(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=3, bias=True)
)

In [24]:
# 샘플 입력값
x = torch.rand(1, 4)
x

tensor([[0.4254, 0.2607, 0.5908, 0.5233]])

In [25]:
out = model(x)
out

tensor([[ 0.0103, -0.1789,  0.1640]], grad_fn=<AddmmBackward0>)

#### 손실함수/ 옵티마이저 설정

In [45]:
# 분류(마지막 출력 3개) CrossEntropyLoss - 
criterion = nn.CrossEntropyLoss()

# 옵티마이저(Adam 권장)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)   # lr(learning rate)학습률 : 보통 0.01 ~ 0.001 사이로 지정

#### 1회 Epoch 학습

In [27]:
y = torch.tensor([1])

# Foward
# 예측
outputs = model(x)
# 손실계산
loss = creterion(outputs, y)

# Backward(역전파) : 뉴런을 통과한 값이 다시 다음 훈련의 입력으로 사용
# 기울기(가중치) 초기화
optimizer.zero_grad()   # 이전 단계에서 계산된 gradient(기울기/가중치) 초기화
# 역전파 계산
loss.backward()     # 손실 기준으로 각 파라미터에 대한 gradient 계산(오차 역전파)
# 파라미터 업데이트
optimizer.step()      # 계산한 gradient를 이용해서 파라미터 업데이트(학습훈련)

# 현재 손실 값을 출력
loss.item()

1.285706877708435

### 데이터셋, 데이터로더
- `Dataset`, `DataLoaer` 학습
- 내장 데이터셋
- 커스텀 데이터셋 만들기
- 배치 학습 처리

#### 데이터셋, 데이터로더란?
- Dataset - 데이터를 불러오는 방법
- DataLoader - 데이터를 배치 단위로 나누기, 셔플, 병렬처리 방법

#### 추가
- 머신러닝, 딥러닝에 사용되는 데이터샘플 종류 예제

In [28]:
from sklearn.datasets import load_breast_cancer, load_wine      # 11개 데이터셋
import seaborn as sns

datas = sns.load_dataset('titanic')    # https://github.com/mwaskom/seaborn-data 에서 확인

from torchvision.datasets import FashionMNIST, MNIST, CIFAR10   # 이미지 데이터셋
from keras.datasets import mnist, fashion_mnist, boston_housing, cifar10, cifar100, imdb    # 이미지, 텍스트 데이터셋

In [29]:
datas

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


#### 붓꽃(Iris) 데이터세 사용

In [30]:
# 데이터셋 처리 모듈 로드
from sklearn.datasets import load_iris, load_wine
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader

In [31]:
# 데이터 로드
iris = load_iris()

# 입력(특성)데이터 X, 타겟 y == input, target
X = iris['data']
y = iris['target']

print(X.shape, y.shape)     # numpy 데이터

(150, 4) (150,)


In [32]:
# sepal(꽃받침) length(cm), sepal width(cm), petal(꽃잎) length(cm), petal width(cm)
X[0, : ]

array([5.1, 3.5, 1.4, 0.2])

In [33]:
# 0(setosa), 1(versicolor), 2(virginica)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [34]:
# 표준화(정규화)
scaler = StandardScaler()   # 사이킷런의 표준정규화 클래스 사용
X = scaler.fit_transform(X)

In [35]:
X[0, :]     # 이전 array([5.1, 3.5, 1.4, 0.2])

array([-0.90068117,  1.01900435, -1.34022653, -1.3154443 ])

In [36]:
# 훈련/테스트세트 분리
train_scaled, test_scaled, train_target, test_target = train_test_split(X, y, test_size=0.2, random_state=42)

In [37]:
# 훈련/검증세트 분리
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [38]:
print(train_scaled.shape, val_scaled.shape, test_scaled.shape)

(96, 4) (24, 4) (30, 4)


In [39]:
train_target

array([2, 1, 1, 0, 2, 2, 2, 0, 1, 2, 2, 1, 2, 1, 1, 0, 0, 1, 2, 0, 0, 2,
       2, 1, 2, 1, 1, 0, 2, 0, 1, 2, 0, 2, 0, 0, 2, 0, 2, 1, 2, 0, 0, 0,
       0, 2, 0, 0, 1, 2, 1, 0, 1, 2, 1, 2, 2, 2, 2, 1, 1, 0, 0, 0, 2, 0,
       0, 0, 2, 1, 2, 1, 2, 1, 1, 0, 0, 2, 1, 0, 2, 1, 2, 1, 2, 1, 1, 2,
       1, 1, 0, 2, 0, 1, 0, 0])

#### 커스텀 데이터셋 만들기

In [40]:
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)   # 특성(입력)값은 실수
        self.y = torch.tensor(y, dtype=torch.long)      # CorssEntropyLoss에서는 long타입 필요

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
        

- 데이터셋으로 묶고, 데이터로더로 묶기

In [41]:
# 커스텀 데이터셋으로 생성 - 입력(특성)과 타겟을 하나로 묶음
train_dataset = IrisDataset(train_scaled, train_target)
val_dataset = IrisDataset(val_scaled, val_target)

In [42]:
# 데이터로더로 생성 - 배치학습, 셔플, 병렬 데이터 로딩
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)

- 실제 모델 훈련에서는 DataLoader를 사용

#### 배치 학습 반복

In [47]:
i = 1
for batch_input, batch_target in train_loader: 
    outputs = model(batch_input)
    loss = criterion(outputs, batch_target)

    print(f'Batch loss : {loss.item()} / {i}')    
    i = i + 1

Batch loss : 1.2507779598236084 / 1
Batch loss : 1.0403858423233032 / 2
Batch loss : 1.1889450550079346 / 3
Batch loss : 1.2533055543899536 / 4
Batch loss : 1.0674985647201538 / 5
Batch loss : 1.3319851160049438 / 6
Batch loss : 1.118078351020813 / 7
Batch loss : 1.1592073440551758 / 8
Batch loss : 1.2136571407318115 / 9
Batch loss : 1.0668302774429321 / 10
Batch loss : 1.1347163915634155 / 11
Batch loss : 1.2994091510772705 / 12


- 96개 입력을 16으로 나눔: 배치사이즈가 16이니까 6번 epoch

### 학습루프 작성
- 전체 학습 루프 구성
- Epoch, Batch 사용
- train(), evalutate() 확인

#### 신경망 클래스 정의

In [49]:
# 아이리스용 신경망
class IrisNet(nn.Module):
    def __init__(self):
        super(IrisNet, self).__init__()
        self.fc1 = nn.Linear(4, 16)     # fc1 : Full Connected Layer / tensorflow Dense()와 동일
        self.fc2 = nn.Linear(16, 3)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### 데이터 준비

In [50]:
# 커스텀 데이터셋
class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    
iris = load_iris()
X = StandardScaler().fit_transform(iris.data)
y = iris.target

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Dataset, DataLoader 합쳐서 생성
train_loader = DataLoader(IrisDataset(X_train, y_train), batch_size=8, shuffle=True)
val_loader = DataLoader(IrisDataset(X_val, y_val), batch_size=8)


#### 모델 디바이스 전달

In [51]:
model = IrisNet().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [52]:
model

IrisNet(
  (fc1): Linear(in_features=4, out_features=16, bias=True)
  (fc2): Linear(in_features=16, out_features=3, bias=True)
)

In [53]:
criterion

CrossEntropyLoss()

In [54]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)

#### 훈련루프 함수 / 평가함수

In [59]:
# 학습루프 함수
def train(model, dataloader, crierion, optimizer):
    model.train()   # 훈련모드
    total_loss = 0

    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    return avg_loss

In [62]:
# 평가함수
def evaluate(model, dataloader, criterion):
    model.eval()  # 평가모드
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():   # 평가시에는 그레디언트 꺼줌
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()

            # 정확도
            preds = torch.argmax(outputs, dim=1)  # 제일 확률이 큰 값의 인덱스 (0, 1, 2)중 하나 값 선택
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    avg_loss = total_loss / len(dataloader)
    accuracy = correct / total
    return avg_loss, accuracy

#### 훈련 실행

In [63]:
EPOCH = 20

for epoch in range(EPOCH):
    train_loss = train(model, train_loader, criterion, optimizer)
    val_loss, val_acc = evaluate(model, val_loader, criterion)

    print(f'[{epoch+1}/{EPOCH}] Train Loss: {train_loss:.3f} | Val Loss: {val_loss:.3f}, Val Accuracy: {val_acc:.2%}')

[1/20] Train Loss: 0.260 | Val Loss: 0.162, Val Accuracy: 96.67%
[2/20] Train Loss: 0.198 | Val Loss: 0.117, Val Accuracy: 100.00%
[3/20] Train Loss: 0.152 | Val Loss: 0.093, Val Accuracy: 100.00%
[4/20] Train Loss: 0.124 | Val Loss: 0.076, Val Accuracy: 100.00%
[5/20] Train Loss: 0.110 | Val Loss: 0.073, Val Accuracy: 96.67%
[6/20] Train Loss: 0.102 | Val Loss: 0.066, Val Accuracy: 100.00%
[7/20] Train Loss: 0.092 | Val Loss: 0.061, Val Accuracy: 96.67%
[8/20] Train Loss: 0.090 | Val Loss: 0.055, Val Accuracy: 100.00%
[9/20] Train Loss: 0.079 | Val Loss: 0.050, Val Accuracy: 100.00%
[10/20] Train Loss: 0.074 | Val Loss: 0.054, Val Accuracy: 96.67%
[11/20] Train Loss: 0.075 | Val Loss: 0.044, Val Accuracy: 100.00%
[12/20] Train Loss: 0.073 | Val Loss: 0.044, Val Accuracy: 100.00%
[13/20] Train Loss: 0.067 | Val Loss: 0.049, Val Accuracy: 96.67%
[14/20] Train Loss: 0.071 | Val Loss: 0.050, Val Accuracy: 96.67%
[15/20] Train Loss: 0.078 | Val Loss: 0.042, Val Accuracy: 100.00%
[16/20] Tr

- CUDA로 0.7s